In [1]:
# Using the nba api from https://github.com/swar/nba_api?tab=readme-ov-file
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.endpoints import boxscoreadvancedv2
from nba_api.stats.endpoints import boxscoretraditionalv2
import pandas as pd
from json import loads, dumps
import time
from datetime import datetime, timedelta,date

In [2]:



today = date.today()
daterange=today-timedelta(1)

In [3]:
#Getting the game logs from yesterday's game
gamelog=leaguegamelog.LeagueGameLog(date_from_nullable=daterange,date_to_nullable=daterange)

In [4]:
gamelogdf=gamelog.get_data_frames()[0]
gamelogdf

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22024,1610612754,IND,Indiana Pacers,0022401216,2024-12-15,IND vs. NOP,W,240,41,...,36,43,33,10,9,17,21,119,15,1
1,22024,1610612740,NOP,New Orleans Pelicans,0022401216,2024-12-15,NOP @ IND,L,240,38,...,30,49,18,11,4,18,19,104,-15,1
2,22024,1610612764,WAS,Washington Wizards,0022401217,2024-12-15,WAS vs. BOS,L,240,40,...,34,51,22,10,7,16,25,98,-14,1
3,22024,1610612738,BOS,Boston Celtics,0022401217,2024-12-15,BOS @ WAS,W,240,40,...,40,56,26,11,5,18,15,112,14,1
4,22024,1610612752,NYK,New York Knicks,0022401227,2024-12-15,NYK @ ORL,W,240,33,...,38,45,20,8,3,19,16,100,9,1
5,22024,1610612753,ORL,Orlando Magic,0022401227,2024-12-15,ORL vs. NYK,L,240,36,...,29,39,24,8,10,15,26,91,-9,1
6,22024,1610612759,SAS,San Antonio Spurs,0022401218,2024-12-15,SAS vs. MIN,L,240,37,...,38,50,34,6,8,19,18,92,-14,1
7,22024,1610612750,MIN,Minnesota Timberwolves,0022401218,2024-12-15,MIN @ SAS,W,240,40,...,37,51,26,11,4,12,13,106,14,1
8,22024,1610612756,PHX,Phoenix Suns,0022401219,2024-12-15,PHX vs. POR,W,240,43,...,34,46,23,7,2,11,13,116,7,1
9,22024,1610612757,POR,Portland Trail Blazers,0022401219,2024-12-15,POR @ PHX,L,240,42,...,36,48,30,6,4,13,22,109,-7,1


In [5]:
#list of Game Id
gameidlist = list(set(gamelogdf['GAME_ID']))

In [6]:
# Pulling the advanced box score from each game id
allboxdf=pd.DataFrame()
n = 30

for i in range(len(gameidlist)):
    boxscore=boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=gameidlist[i])
    bx=boxscore.get_data_frames()[1]
    allboxdf=pd.concat([allboxdf,bx])
    if (i + 1) % n == 0:
        time.sleep(20)
    

    

In [7]:
#Combining the game logs and the advanced box scores
df_merged = pd.merge(allboxdf,
                    gamelogdf[['GAME_ID','TEAM_ID','GAME_DATE','PTS']],
                    on=['GAME_ID','TEAM_ID'],how='right')

In [8]:
df_merged=df_merged.copy()
df_merged['GAME_DATE']=df_merged['GAME_DATE'].apply(pd.to_datetime)

In [9]:

#swapping rows so that we can get the Opponents Pts and Poss for each game a team plays
def swap_rows(df):
    for i in range(0, len(df), 2):
        if i + 1 < len(df):
            df.iloc[i], df.iloc[i + 1] = df.iloc[i + 1].copy(), df.iloc[i].copy()
    return df


In [10]:
df_mergedsort=df_merged.sort_values(by=['GAME_DATE','GAME_ID'])

In [11]:
df_mergedsort.reset_index(inplace=True,drop=True)

In [12]:
df_opp=df_mergedsort[['TEAM_ID','PTS','POSS','TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CITY']]


In [13]:
df_opp=df_opp.copy()

df_swap=swap_rows(df_opp)

In [14]:
df_swap.rename(columns={'TEAM_ID': 'OPP_TEAM_ID', 'PTS': 'OPP_PTS','POSS': 'OPP_POSS','TEAM_NAME': 'OPP_TEAM_NAME','TEAM_ABBREVIATION':'OPP_TEAM_ABBERVIATION','TEAM_CITY':'OPP_TEAM_CITY'}, inplace=True)

In [15]:
dfall= pd.concat([df_mergedsort ,df_swap], axis=1)

In [16]:
filename = "filepath "+f"{daterange}.csv"

dfall.to_csv(filename,index=False)